In [1]:
import pandas as pd

In [2]:
import json
#read json
with open('data/interview_date_training_data.json') as f:
    data = json.load(f)


In [3]:
df = pd.DataFrame(columns=['transcript', 'conversation_length'])
for r in data:
    dat = r['sample_input'].split('\n###\nTRANSCRIPT\n')[-1][:-4]
    dat = dat.split('\n')

    conv = []
    for dia in dat:
        # check if a string startswith 'Agent' 
        if dia.startswith('Agent:'):
            conv.append({'Agent':dia[7:]})
        elif dia.startswith('Customer:'):
            conv.append({'Customer':dia[10:]})
        else:
            conv.append({'Other':dia})

    # append row
    row = {'transcript': conv, 'conversation_length': len(conv)}
    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)

df

,transcript,conversation_length
0,[{'Customer': 'no ma'am you're not this is gar...,2
1,"[{'Agent': 'Hi, I'm Taylor, calling from Westl...",3
2,"[{'Agent': 'Hi, I'm Taylor, calling from Westl...",2
3,"[{'Agent': 'Hi, I'm Taylor, calling from Westl...",2
4,[{'Agent': 'When do you think you could make a...,2
...,...,...
108,[{'Agent': 'When would you be able to make a p...,2
109,[{'Agent': 'Is it possible for you to make the...,2
110,[{'Agent': 'When do you plan to make the payme...,2
111,[{'Agent': 'Can you advise when you'll manage ...,2


In [4]:
df.to_csv('data/interview_date_training_data.csv', index=False)

In [5]:
df['conversation_length'].value_counts()

conversation_length
2    110
3      2
4      1
Name: count, dtype: int64

In [6]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_lg
nlp = en_core_web_lg.load()

In [7]:
sentence = str(data[8]['sample_input'].split('\n###\nTRANSCRIPT\n')[-1][:-4])
print("TRANSCRIPT:\n", sentence, '\n')

doc = nlp(sentence)
print([(X.text, X.label_) for X in doc.ents if X.label_ == 'DATE'])

TRANSCRIPT:
 Agent: Hi, I'm Taylor, calling from Westlake Financial on a recorded line. How can I assist you today?
Customer: I received a call from this number, but I am not sure why. Can you help me understand why I am being contacted? 

[('today', 'DATE')]


In [8]:
for i, r in df.iterrows():

    transcript = r['transcript']
    print("TRANSCRIPT:\n")
    formatted_transcript = ""
    for ele in transcript:
        for e in ele.keys():
            # print(e, ': ', ele[e], '\n')
            if e == 'Agent' and ele[e].startswith("Hi, I'm Taylor, calling from Westlake Financial on a recorded line"):
                continue
            formatted_transcript += e + ': '  + ele[e] + '\n'

    print(formatted_transcript)
    doc = nlp(formatted_transcript)

    print([(X.text, X.label_) for X in doc.ents])
    if i==10:
        break

TRANSCRIPT:

Customer: no ma'am you're not this is gary dinger's number please remove it from your calling list
Agent: My apologies for the misunderstanding. Have a great rest of your day. Goodbye!

[("gary dinger's", 'PERSON')]
TRANSCRIPT:

Customer: for what excuse me
Agent: You have a loan with us for a 2015 Ram Promaster City. The monthly payment of $290.94 is overdue. Can you make a payment today?

[('2015', 'DATE'), ('monthly', 'DATE'), ('290.94', 'MONEY'), ('today', 'DATE')]
TRANSCRIPT:

Customer: i get ma'am i was like hold on i i was wondering if we can do like a payment alright payment arrangement because i have half of the money i just need a couple days to get the rest

[('half', 'CARDINAL'), ('a couple days', 'DATE')]
TRANSCRIPT:

Customer: yeah i did i did it through text but that that bring the house for this one and then press two or whatever

[('two', 'CARDINAL')]
TRANSCRIPT:

Agent: When do you think you could make a payment?
Customer: I'm sorry, there's been a misund

In [9]:
# from transformers import OpenAIGPTTokenizer, OpenAIGPTModel
# import torch

# tokenizer = OpenAIGPTTokenizer.from_pretrained("openai-gpt")
# model = OpenAIGPTModel.from_pretrained("openai-gpt")

# inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
# outputs = model(**inputs)

# last_hidden_states = outputs.last_hidden_state

In [10]:
# outputs

In [11]:
# token_ids = last_hidden_states.view(-1).tolist()

# # Decode the token_ids into a sentence
# output_sentence = tokenizer.decode(token_ids, skip_special_tokens=True)

# print(output_sentence)

In [12]:
# output_sentence = tokenizer.decode(last_hidden_states[0], skip_special_tokens=True)

# print(output_sentence)

In [13]:
# from transformers import OpenAIGPTTokenizer, OpenAIGPTLMHeadModel

# # Load the pre-trained model and tokenizer
# tokenizer = OpenAIGPTTokenizer.from_pretrained("openai-gpt")
# model = OpenAIGPTLMHeadModel.from_pretrained("openai-gpt")

# # Define the input text
# input_text = "What id your name"

# # Tokenize the input text
# input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# # Generate text based on the input
# output = model.generate(input_ids, max_length=50, num_return_sequences=1, no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0.7)

# # Decode the generated output
# generated_text = tokenizer.decode(output[0], skip_special_tokens=True)




In [14]:
txt = "Agent: Can you make a payment on the 15th of this month?\n Customer: Can we move it to the 20th instead? Given the above transcript and todays date as 01/01/2023, give me the date when the customer is expected to make their payment in the format dd/mm/yyyy. Just output the date in your response and nothing else."


In [16]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl")

input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

/Users/pratt/miniconda3/envs/ml_base/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:41<00:00, 20.83s/it]
/Users/pratt/miniconda3/envs/ml_base/lib/python3.11/site-packages/transform

<pad> Wie alt sind Sie?</s>
